# Clean Network

Cleans the network files to be only what is specified in the data model so as not to store extraneous data / minimize space.

## Roadway

In [54]:
from pathlib import Path

from network_wrangler import load_roadway_from_dir, load_transit, write_roadway, write_transit
from network_wrangler.models.gtfs.tables import (
    WranglerFrequenciesTable,
    WranglerShapesTable,
    WranglerStopsTable,
    WranglerStopTimesTable,
    WranglerTripsTable,
    RoutesTable,
)
from network_wrangler.models.roadway.tables import RoadLinksTable, RoadNodesTable, RoadShapesTable

In [43]:
def shared_fields_w_datamodel(df, model):
    """Returns a list of fields that are in both the dataframe and the datamodel."""
    model_fields = list(model.__fields__.keys())
    df_fields = list(df.columns)
    return list(set(model_fields).intersection(df_fields))

In [ ]:
og_net = load_roadway_from_dir(Path.cwd())

In [22]:
links_data_model_fields = list(RoadLinksTable.__fields__.keys())
shared_fields = set(links_data_model_fields).intersection(set(og_net.links_df.columns))
DEL_FIELDS = ["locationReferences"]
KEEP_FIELDS = ["geometry"]
specified_fields = ["model_link_id", "A", "B", "name", "osm_link_id"]
remaining_fields = list(
    shared_fields.union(set(KEEP_FIELDS)) - set(DEL_FIELDS) - set(specified_fields)
)
final_fields = specified_fields + remaining_fields
og_net.links_df = og_net.links_df[final_fields]

In [27]:
nodes_data_model_fields = list(RoadNodesTable.__fields__.keys())
shared_node_fields = list(set(nodes_data_model_fields).intersection(set(og_net.nodes_df.columns)))
og_net.nodes_df = og_net.nodes_df[shared_node_fields]

In [32]:
shapes_data_model_fields = list(RoadShapesTable.__fields__.keys())
shared_shapes_fields = list(
    set(shapes_data_model_fields).intersection(set(og_net.shapes_df.columns))
)
og_net.shapes_df = og_net.shapes_df[shared_shapes_fields]

In [36]:
write_roadway(og_net, Path.cwd() / "smaller")

In [37]:
smaller_net = load_roadway_from_dir(Path.cwd() / "smaller")
smaller_net.links_df

## Transit

In [ ]:
og_transit = load_transit(Path.cwd())

In [49]:
sh_f = shared_fields_w_datamodel(og_transit.feed.shapes, WranglerShapesTable)
og_transit.feed.shapes = og_transit.feed.shapes[sh_f]

In [51]:
sh_f = shared_fields_w_datamodel(og_transit.feed.trips, WranglerTripsTable)
og_transit.feed.trips = og_transit.feed.trips[sh_f]

In [63]:
write_transit(og_transit, Path.cwd() / "smaller")

In [ ]:
smaller_transit = load_transit(Path.cwd() / "smaller")